In [42]:
from xml.etree import ElementTree
from xml.dom import minidom
from xml.etree.ElementTree import Element, SubElement, Comment, indent

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, encoding="ISO-8859-1")
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [66]:
import numpy as np
import os

valve_start = 1

hyb_start = 1

reg_start = 2

num_rounds = 13

add_ref_hyb = True

data_type = 'R'

valve_ids = np.arange(valve_start, valve_start + num_rounds)
hyb_ids = np.arange(hyb_start, hyb_start + num_rounds)
reg_names = [f'{data_type}{_i}' for _i in np.arange(reg_start, reg_start + num_rounds)]

In [67]:
source_folder = r'D:\Pu\20211216-P_brain_CTP12-RNA'
target_drive = r'\\KOLMOGOROV\Chromatin_NAS_1'
imaging_protocol = r'Conv_Zscan_750_647_561_s50_n250_15Hz'
ref_imaging_protocol = r'Conv_Zscan_750_647_561_488_405_s50_n250_15Hz'
ref_hyb = 0

In [68]:
cmd_seq = Element('command_sequence')

if add_ref_hyb:
    # add hyb 0
    # comments
    comment = Comment(f"Hyb 0")
    cmd_seq.append(comment)
    # flow imaging buffer
    imaging = SubElement(cmd_seq, 'valve_protocol')
    imaging.text = f"Flow Imaging Buffer"

    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H0{data_type}1")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"

    # imaging loop
    _im_p = ref_imaging_protocol

    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=_im_p)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "2000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text 
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"


for _i, (_vid, _hid, _rname) in enumerate(zip(valve_ids, hyb_ids, reg_names)):
    
    # select protocol
    if _hid == ref_hyb:
        _im_p = ref_imaging_protocol
    else:
        _im_p = imaging_protocol
    
    if _hid > 0:
        # TCEP
        tcep = SubElement(cmd_seq, 'valve_protocol')
        tcep.text = "Flow TCEP"
        # wash tcep
        tcep_wash = SubElement(cmd_seq, 'valve_protocol')
        tcep_wash.text = "Flow 2xSSC"
        
    # comments
    comment = Comment(f"Hyb {_hid} with {_vid} for {_rname}")
    cmd_seq.append(comment)
    # flow adaptor
    adt = SubElement(cmd_seq, 'valve_protocol')
    adt.text = f"Hybridize {_vid}"
    # delay time
    adt_incubation = SubElement(cmd_seq, 'valve_protocol')
    adt_incubation.text = f"Incubate 15min"
    # wash
    wash = SubElement(cmd_seq, 'valve_protocol')
    wash.text = "Flow Wash Buffer"
    
    # flow imaging buffer
    imaging = SubElement(cmd_seq, 'valve_protocol')
    imaging.text = f"Flow Imaging Buffer"
    
    # change directory
    change_dir = SubElement(cmd_seq, 'change_directory')
    change_dir.text = os.path.join(source_folder, f"H{_hid}{_rname.upper()}")
    # wakeup
    wakeup = SubElement(cmd_seq, 'wakeup')
    wakeup.text = "5000"
    
    # imaging loop
    loop = SubElement(cmd_seq, 'loop', name='Position Loop Zscan', increment="name")
    loop_item = SubElement(loop, 'item', name=_im_p)
    loop_item.text = " "
    # delay time
    delay = SubElement(cmd_seq, 'delay')
    delay.text = "2000"
    # copy folder
    copy_dir = SubElement(cmd_seq, 'copy_directory')
    source_dir = SubElement(copy_dir, 'source_path')
    source_dir.text = change_dir.text#cmd_seq.findall('change_directory')[-1].text
    target_dir = SubElement(copy_dir, 'target_path')
    target_dir.text = os.path.join(target_drive, 
                                   os.path.basename(os.path.dirname(source_dir.text)), 
                                   os.path.basename(source_dir.text))
    del_source = SubElement(copy_dir, 'delete_source')
    del_source.text = "True"
    # empty line
    indent(target_dir)

ssc = SubElement(cmd_seq, 'valve_protocol')
ssc.text = f"Flow 2xSSC"

final_str = prettify(cmd_seq)
    
print( final_str )

<?xml version="1.0" ?>
<command_sequence>
	<!--Hyb 0-->
	<valve_protocol>Flow Imaging Buffer</valve_protocol>
	<change_directory>D:\Pu\20211216-P_brain_CTP12-RNA\H0R1</change_directory>
	<wakeup>5000</wakeup>
	<loop name="Position Loop Zscan" increment="name">
		<item name="Conv_Zscan_750_647_561_488_405_s50_n250_15Hz"> </item>
	</loop>
	<delay>2000</delay>
	<copy_directory>
		<source_path>D:\Pu\20211216-P_brain_CTP12-RNA\H0R1</source_path>
		<target_path>\\KOLMOGOROV\Chromatin_NAS_1\20211216-P_brain_CTP12-RNA\H0R1</target_path>
		<delete_source>True</delete_source>
	</copy_directory>
	<valve_protocol>Flow TCEP</valve_protocol>
	<valve_protocol>Flow 2xSSC</valve_protocol>
	<!--Hyb 1 with 1 for R2-->
	<valve_protocol>Hybridize 1</valve_protocol>
	<valve_protocol>Incubate 15min</valve_protocol>
	<valve_protocol>Flow Wash Buffer</valve_protocol>
	<valve_protocol>Flow Imaging Buffer</valve_protocol>
	<change_directory>D:\Pu\20211216-P_brain_CTP12-RNA\H1R2</change_directory>
	<wakeup>5000</

# save this xml

In [69]:
save_filename = os.path.join(source_folder, f"generated_dave_H{hyb_start}-{hyb_start+num_rounds-1}.txt")
with open(save_filename, 'w') as _output_handle:
    print(save_filename)
    _output_handle.write(final_str)
    

D:\Pu\20211216-P_brain_CTP12-RNA\generated_dave_H1-13.txt
